<a href="https://colab.research.google.com/github/SaumilShah-7/Airbus-Ship-Detection-Challenge-Kaggle/blob/master/Airbus_Ship_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import seaborn as sns
from sklearn.model_selection import train_test_split

TensorFlow 2.x selected.
2.0.0


In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c airbus-ship-detection

  0% 0.00/274k [00:00<?, ?B/s]
100% 274k/274k [00:00<00:00, 83.4MB/s]
 50% 9.00M/18.0M [00:00<00:00, 18.0MB/s]
100% 18.0M/18.0M [00:00<00:00, 36.6MB/s]
100% 2.12G/2.12G [00:31<00:00, 118MB/s]
100% 2.12G/2.12G [00:31<00:00, 71.7MB/s]
100% 26.4G/26.4G [04:18<00:00, 97.4MB/s]
100% 26.4G/26.4G [04:18<00:00, 110MB/s] 


In [0]:
!unzip -q train_v2.zip -d train
!unzip -q train_ship_segmentations_v2.csv.zip -d train_ship_segmentations
!unzip -q test_v2.zip -d test

!rm -rf train_v2.zip
!rm -rf train_ship_segmentations_v2.csv.zip
!rm -rf test_v2.zip

In [0]:
df = pd.read_csv('train_ship_segmentations/train_ship_segmentations_v2.csv')

print(len(df['ImageId'].unique()), len([name for name in os.listdir('train')]))

df['Count'] = df['EncodedPixels'].notnull()*1
df_h = df.groupby(['ImageId'])['Count'].sum()

print(pd.concat([df_h.value_counts(), df_h.value_counts(normalize=True)], axis=1))
sns.countplot(df_h)

In [0]:
df_u = pd.DataFrame({'ImageId': df_h.index, 'Shipnum': df_h.values})
df_u['Filesize'] = df_u['ImageId'].apply(lambda x: os.path.getsize('train/'+x)/1024)

print(df_u.shape[0])
df_u = df_u[df_u['Filesize'] > 50]
print(df_u.shape[0])

# import shutil
# os.makedirs('50-60')
# for index, row in df_u.iterrows():
#   if row['Filesize'] < 60 and row['Filesize'] >= 50:
#     shutil.copy('train/'+row['ImageId'], '50-60')
#   else:
#     pass
  
# !zip -r 50-60.zip 50-60
# !rm -rf 50-60

In [0]:
image_height = 768
image_width = 768

def rle_decode(img_id):
  img_rle = [*df.loc[df['ImageId']==img_id]['EncodedPixels']]
  mask = np.zeros(image_height*image_width, dtype=np.uint8)
  for x in img_rle:
    x = x.split(' ')
    for i in range(0, len(x), 2):
      mask[int(x[i]):int(x[i])+int(x[i+1])]=255
  return Image.fromarray(mask.reshape(image_height, image_width).astype(np.uint8).T)

In [0]:
from IPython.display import display

# print(df_u[df_u==5])

img = Image.open('train/000194a2d.jpg')
display(img)

img1 = rle_decode('000194a2d.jpg')
display(img1)

In [0]:
train_df, test_df = train_test_split(df_u, test_size=0.2, stratify=df_u['Shipnum'])

print(train_df.shape[0], test_df.shape[0])

In [0]:
if not os.path.exists('mask'):
  os.makedirs('mask')

for name in os.listdir('train'):
  if df_h[name]!=0:
    img = rle_decode(name)
    img.save('mask/'+name, 'JPEG')